<a href="https://colab.research.google.com/github/siperolli/IBM-Data-Science-Capstone-/blob/main/Final_Capstone_bsb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3><b> IBM Applied Data Science Capstone Course by Coursera
Week 5 Final Report</b> </H3>

What is the best business in Brasília, Brazil based on consumer tastes as measured by the booking of places that are most frequented by Forsquare and by income?



1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import requests
import urllib
from bs4 import BeautifulSoup
import re

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



2. Scrap data from Wikipedia page into a DataFrame

In [2]:
url = "https://pt.wikipedia.org/wiki/Lista_de_regi%C3%B5es_administrativas_do_Distrito_Federal_por_renda_per_capita"
r = requests.get(url)
# convert url em text
data = r.text
#convert 
soup = BeautifulSoup(data, 'html.parser')
tb=soup.find('table').text

In [3]:
tb

'\n\n\n\nPosição\n\nRegião administrativa\n\nRenda per capita\n\n\n1\n\n Lago Sul\n\n8.317,40\n\n\n2\n\n Sudoeste/Octogonal\n\n7.073,70\n\n\n3\n\n Plano Piloto\n\n6.778,00\n\n\n4\n\n Lago Norte\n\n6.362,30\n\n\n5\n\n Park Way\n\n5.902,90\n\n\n6\n\n Jardim Botânico\n\n5.861,90\n\n\n7\n\n Águas Claras\n\n4.407,50\n\n\n8\n\n SIA\n\n3.815,20\n\n\n9\n\n Cruzeiro\n\n3.754,80\n\n\n10\n\n Guará\n\n3.642,60\n\n\n11\n\n Vicente Pires\n\n2.686,60\n\n\n12\n\n Núcleo Bandeirante\n\n2.381,10\n\n\n13\n\n Sobradinho II\n\n2.362,20\n\n\n14\n\n Taguatinga\n\n2.206,60\n\n\n15\n\n Sobradinho\n\n2.192,30\n\n\n16\n\n Gama\n\n1.596,40\n\n\n17\n\n Candangolândia\n\n1.415,70\n\n\n18\n\n São Sebastião\n\n1.351,20\n\n\n19\n\n Riacho Fundo\n\n1.310,40\n\n\n20\n\n Planaltina\n\n1.140,20\n\n\n21\n\n Brazlândia\n\n1.118,90\n\n\n22\n\n Ceilândia\n\n1.116,10\n\n\n23\n\n Samambaia\n\n992.30\n\n\n24\n\n Santa Maria\n\n977,00\n\n\n25\n\nItapoã\n\n925,80\n\n\n26\n\n Recanto das Emas\n\n857,60\n\n\n27\n\n Varjão\n\n834,20\

In [4]:
inc=pd.DataFrame(re.split('\n{3,5}', tb))



In [5]:
inc

,0
0,
1,Posição\n\nRegião administrativa\n\nRenda per ...
2,"1\n\n Lago Sul\n\n8.317,40"
3,"2\n\n Sudoeste/Octogonal\n\n7.073,70"
4,"3\n\n Plano Piloto\n\n6.778,00"
5,"4\n\n Lago Norte\n\n6.362,30"
6,"5\n\n Park Way\n\n5.902,90"
7,"6\n\n Jardim Botânico\n\n5.861,90"
8,"7\n\n Águas Claras\n\n4.407,50"
9,"8\n\n SIA\n\n3.815,20"


In [6]:
df=inc[0].str.split('\\n', expand=True)

In [7]:
df

,0,1,2,3,4,5
0,,None,None,None,None,None
1,Posição,,Região administrativa,,Renda per capita,None
2,1,,Lago Sul,,"8.317,40",None
3,2,,Sudoeste/Octogonal,,"7.073,70",None
4,3,,Plano Piloto,,"6.778,00",None
5,4,,Lago Norte,,"6.362,30",None
6,5,,Park Way,,"5.902,90",None
7,6,,Jardim Botânico,,"5.861,90",None
8,7,,Águas Claras,,"4.407,50",None
9,8,,SIA,,"3.815,20",None


In [8]:
df.drop(columns=[0,1,3,5], inplace=True)
df.drop([0], axis=0, inplace=True)
df.drop([1], axis=0, inplace=True)

In [9]:
df.columns = ["City", "PCI"]

Plano Piloto é constituindo de Asa Norte e Asa Sul. Por isso foram separados para melhor distribuição dos locais que será visto mais abaixo.
A Renda per capita do Plano Piloto foi a mesma para os 2 tendo em vista que é renda aproximada. 

In [10]:
 y1=['Asa Norte', '6.778,00']
 y2=['Asa Sul', '6.778,00']

In [11]:
df=df.append({'City' : 'Asa Norte', 'PCI' : '6.778,00'} , ignore_index=True)
df=df.append({'City' : 'Asa Sul', 'PCI' : '6.778,00'} , ignore_index=True)

In [12]:
df.drop([2], axis=0, inplace=True)

In [13]:
df=df.reset_index(drop=True)

In [14]:
df

,City,PCI
0,Lago Sul,"8.317,40"
1,Sudoeste/Octogonal,"7.073,70"
2,Lago Norte,"6.362,30"
3,Park Way,"5.902,90"
4,Jardim Botânico,"5.861,90"
5,Águas Claras,"4.407,50"
6,SIA,"3.815,20"
7,Cruzeiro,"3.754,80"
8,Guará,"3.642,60"
9,Vicente Pires,"2.686,60"


In [15]:
df['PCI'] = df['PCI'].apply(lambda x: float(x.replace(".","").replace(",",".")))

In [16]:
df.replace({99230.0 : 992.30}, inplace=True)

3. Get the geographical coordinates

In [17]:
df['Latitude'] = np.nan
df['Longitude'] = np.nan

In [18]:
for i in range(len(df)):
  address = 'Brasil,'+ 'Distrito Federal,'+ df.iloc[i,0]
  geolocator = Nominatim(user_agent="foursquare_agent")
  location = geolocator.geocode(address, timeout=10)

  latitude = location.latitude
  longitude = location.longitude
  df.iloc[i,2] = latitude
  df.iloc[i,3] = longitude

In [19]:
df

,City,PCI,Latitude,Longitude
0,Lago Sul,8317.4,-15.839182,-47.875534
1,Sudoeste/Octogonal,7073.7,-15.800219,-47.924390
2,Lago Norte,6362.3,-15.734235,-47.864158
3,Park Way,5902.9,-15.902739,-47.963322
4,Jardim Botânico,5861.9,-15.863892,-47.788521
5,Águas Claras,4407.5,-15.841993,-48.028121
6,SIA,3815.2,-15.806182,-47.959158
7,Cruzeiro,3754.8,-15.790782,-47.937443
8,Guará,3642.6,-15.823563,-47.976816
9,Vicente Pires,2686.6,-15.813571,-48.015605


Create a map of cities

In [20]:
# create map using latitude and longitude values
map_df = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_df)  

In [21]:
map_df

4. Use the Foursquare API to explore the cities

Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = 'KRYP05IZOX5R5AVHGFDH1RE53PIVLGT3TRL3DH0Z1EN01R2B' # your Foursquare ID
CLIENT_SECRET = '5Q3K102L002VIIMRSBCR0VC3GS1QVACBHI0TFWAWAVQMFWUU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KRYP05IZOX5R5AVHGFDH1RE53PIVLGT3TRL3DH0Z1EN01R2B
CLIENT_SECRET:5Q3K102L002VIIMRSBCR0VC3GS1QVACBHI0TFWAWAVQMFWUU


Note:

Marco Zero is a place that marks the beginning of the construction of Brasília, which happened with a crossing of roads.

Currently it is not demarcated by signs or monuments, since there is a building built on it: The "Rodoviária do Plano Piloto"

The coordinates are:

Lat = -15.79348666533862

Long = -47.883220160496315

The cities are distant from each other so the big radius for analysis



In [23]:
cp_lat = -15.79348666533862
cp_long = -47.883220160496315
print(cp_lat, cp_long)

-15.79348666533862 -47.883220160496315


**Now, let's get the top 100 venues that are within a radius of 80000 meters.**

In [24]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 80000 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    cp_lat,
    cp_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KRYP05IZOX5R5AVHGFDH1RE53PIVLGT3TRL3DH0Z1EN01R2B&client_secret=5Q3K102L002VIIMRSBCR0VC3GS1QVACBHI0TFWAWAVQMFWUU&v=20180605&ll=-15.79348666533862,-47.883220160496315&radius=80000&limit=100'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '608606b27b4c0b63c41bb83e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b816f4bf964a520cfa530e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4b816f4bf964a520cfa530e3',
       'location': {'address': 'Pq. Recreativo Sarah Kubitschek',
        'cc': 'BR',
        'city': 'Brasília',
        'country': 'Brasil',
        'distance': 2366,
        'formattedAddress': ['Pq. Recreativo Sarah Kubitschek',
         'Brasília, DF',
         '70610-300',
         'Brasil'],
        'lat': -15.797626386075214,
        

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Parque da Cidade Sarah Kubitschek,Park,-15.797626,-47.904888
1,Windsor Brasília,Hotel,-15.790823,-47.885024
2,Pontão do Lago Sul,Pier,-15.825249,-47.872908
3,Unique Family Fitness Club,Gym / Fitness Center,-15.797599,-47.914258
4,Iate Clube de Brasília,Harbor / Marina,-15.779039,-47.860222


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


**Now write the code to run the above function on each city and create a new dataframe called b_venues.**



In [30]:
# type your answer here
b_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

 Lago Sul
 Sudoeste/Octogonal
 Lago Norte
 Park Way
 Jardim Botânico
 Águas Claras
 SIA
 Cruzeiro
 Guará
 Vicente Pires
 Núcleo Bandeirante
 Sobradinho II
 Taguatinga
 Sobradinho
 Gama
 Candangolândia
 São Sebastião
 Riacho Fundo
 Planaltina
 Brazlândia
 Ceilândia
 Samambaia
 Santa Maria
Itapoã
 Recanto das Emas
 Varjão
 Paranoá
 Fercal
 Riacho Fundo II
 SCIA
Asa Norte
Asa Sul


**Let's create a table with all categories reported by FourSquare for each city**

In [31]:
venues_city = b_venues.pivot_table(columns=['Venue Category'], index=['City'], aggfunc='count', fill_value=0)

In [32]:
venues_city.head()

City Latitude                                     \
Venue Category     Acai House Accessories Store Asian Restaurant   
City                                                               
 Brazlândia                 1                 0                0   
 Candangolândia             0                 0                0   
 Ceilândia                  0                 0                1   
 Cruzeiro                   1                 0                0   
 Fercal                     0                 0                0   

                                                                              \
Venue Category  Athletics & Sports Auto Dealership Auto Garage Auto Workshop   
City                                                                           
 Brazlândia                      0               0           0             0   
 Candangolândia                  0               0           0             0   
 Ceilândia                       0               0           0             0   
 Cruzeiro                        0               0           0             0   
 Fercal                          0               0           0             0   

                                                                           \
Venue Category  BBQ Joint Bagel Shop Bakery Bar Beer Bar Bistro Bookstore   
City                                                                        
 Brazlândia             0          0      0   0        0      0         0   
 Candangolândia         0          0      3   0        0      0         0   
 Ceilândia              0          0      0   2        0      0         0   
 Cruzeiro               0          0      2   2        0      0         0   
 Fercal                 0          0      0   0        0      0         0   

                                                                            \
Venue Category  Boutique Bowling Alley Brazilian Restaurant Breakfast Spot   
City                                                                         
 Brazlândia            0             0                    1              0   
 Candangolândia        0             0                    0              0   
 Ceilândia             0             0                    4              0   
 Cruzeiro              0             0                    2              0   
 Fercal                0             0                    0              0   

                                                              \
Venue Category  Brewery Buffet Burger Joint Bus Station Café   
City                                                           
 Brazlândia           0      0            0           0    0   
 Candangolândia       0      0            2           0    0   
 Ceilândia            0      0            3           0    0   
 Cruzeiro             0      0            0           0    0   
 Fercal               0      0            0           0    0   

                                                       \
Venue Category  Cajun / Creole Restaurant Candy Store   
City                                                    
 Brazlândia                             0           0   
 Candangolândia                         0           0   
 Ceilândia                              0           1   
 Cruzeiro                               0           0   
 Fercal                                 0           0   

                                                                             \
Venue Category  Central Brazilian Restaurant Cheese Shop Chinese Restaurant   
City                                                                          
 Brazlândia                                0           0                  0   
 Candangolândia                            0           0                  0   
 Ceilândia                                 0           0                  1   
 Cruzeiro                                  0           0                  0   
 Fercal                                    0           0         

 5. Clustering



How to choose the correct number of clusters?

I will use the silhouette method.

In [33]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

Cluster PCI, Latitude e Longitude


In [34]:
X = df.drop(columns=['City'], axis=1)

Choosing number of clusters

In [35]:
for i in range(2,10):
  kmeans_ = KMeans(n_clusters=i).fit(X)
  labels_ = kmeans_.labels_
  coef = silhouette_score(X,labels_)
  print("N_cluster: {}, score: {}".format(i,coef))

N_cluster: 2, score: 0.7260768601267082
N_cluster: 3, score: 0.6993293113860887
N_cluster: 4, score: 0.7324961005703525
N_cluster: 5, score: 0.7089055929983248
N_cluster: 6, score: 0.708297671688487
N_cluster: 7, score: 0.6297719068804526
N_cluster: 8, score: 0.6142764382908807
N_cluster: 9, score: 0.5963971599813502


In [36]:
#The better k-means clustering
k = 4
kmeans = KMeans(n_clusters=k, random_state=0).fit(X)

Finally, let's visualize the resulting clusters

In [37]:
df_cluster=df.insert(0, 'Cluster Labels', kmeans.labels_)

In [41]:
df_cluster

,Cluster Labels,City,PCI,"(City Latitude, Acai House)","(City Latitude, Accessories Store)","(City Latitude, Asian Restaurant)","(City Latitude, Athletics & Sports)","(City Latitude, Auto Dealership)","(City Latitude, Auto Garage)","(City Latitude, Auto Workshop)","(City Latitude, BBQ Joint)","(City Latitude, Bagel Shop)","(City Latitude, Bakery)","(City Latitude, Bar)","(City Latitude, Beer Bar)","(City Latitude, Bistro)","(City Latitude, Bookstore)","(City Latitude, Boutique)","(City Latitude, Bowling Alley)","(City Latitude, Brazilian Restaurant)","(City Latitude, Breakfast Spot)","(City Latitude, Brewery)","(City Latitude, Buffet)","(City Latitude, Burger Joint)","(City Latitude, Bus Station)","(City Latitude, Café)","(City Latitude, Cajun / Creole Restaurant)","(City Latitude, Candy Store)","(City Latitude, Central Brazilian Restaurant)","(City Latitude, Cheese Shop)","(City Latitude, Chinese Restaurant)","(City Latitude, Chocolate Shop)","(City Latitude, Churrascaria)","(City Latitude, Clothing Store)","(City Latitude, Coffee Shop)","(City Latitude, Comfort Food Restaurant)","(City Latitude, Convenience Store)","(City Latitude, Cosmetics Shop)","(City Latitude, Costume Shop)","(City Latitude, Creperie)","(City Latitude, Cupcake Shop)","(City Latitude, Dance Studio)","(City Latitude, Deli / Bodega)","(City Latitude, Department Store)","(City Latitude, Dessert Shop)","(City Latitude, Diner)","(City Latitude, Dive Bar)","(City Latitude, Donut Shop)","(City Latitude, Drugstore)","(City Latitude, Electronics Store)","(City Latitude, Farmers Market)","(City Latitude, Fast Food Restaurant)","(City Latitude, Fish & Chips Shop)","(City Latitude, Fish Market)","(City Latitude, Flea Market)","(City Latitude, Food Court)","(City Latitude, Food Stand)","(City Latitude, Food Truck)","(City Latitude, Frame Store)","(City Latitude, French Restaurant)","(City Latitude, Fried Chicken Joint)","(City Latitude, Frozen Yogurt Shop)","(City Latitude, Fruit & Vegetable Store)","(City Latitude, Furniture / Home Store)","(City Latitude, Gastropub)","(City Latitude, German Restaurant)","(City Latitude, Goiano Restaurant)","(City Latitude, Gourmet Shop)","(City Latitude, Grocery Store)","(City Latitude, Gym)","(City Latitude, Gym / Fitness Center)","(City Latitude, Hardware Store)","(City Latitude, Hawaiian Restaurant)","(City Latitude, Health & Beauty Service)","(City Latitude, Health Food Store)","(City Latitude, Hookah Bar)","(City Latitude, Hot Dog Joint)","(City Latitude, Hotel)","(City Latitude, IT Services)","(City Latitude, Ice Cream Shop)","(City Latitude, Italian Restaurant)","(City Latitude, Japanese Restaurant)","(City Latitude, Juice Bar)","(City Latitude, Kids Store)","(City Latitude, Lawyer)","(City Latitude, Liquor Store)","(City Latitude, Lottery Retailer)","(City Latitude, Mac & Cheese Joint)","(City Latitude, Market)","(City Latitude, Martial Arts School)","(City Latitude, Massage Studio)","(City Latitude, Metro Station)","(City Latitude, Middle Eastern Restaurant)","(City Latitude, Mobile Phone Shop)","(City Latitude, Motorcycle Shop)","(City Latitude, Movie Theater)","(City Latitude, Newsstand)","(City Latitude, Nightclub)","(City Latitude, Northeastern Brazilian Restaurant)","(City Latitude, Office)","(City Latitude, Other Great Outdoors)","(City Latitude, Paper / Office Supplies Store)","(City Latitude, Pastelaria)","(City Latitude, Pastry Shop)","(City Latitude, Pet Service)","(City Latitude, Pet Store)","(City Latitude, Pharmacy)","(City Latitude, Pilates Studio)","(City Latitude, Pizza Place)","(City Latitude, Playground)","(City Latitude, Plaza)","(City Latitude, Print Shop)","(City Latitude, Recording Studio)","(City Latitude, Rental Car Location)","(City Latitude, Residential Building (Apartment / Condo))","(City Latitude, Restaurant)","(City Latitude, Salad Place)","(City Latitude, Salon / Barbershop)","(City Latitude, Sandwich Place)","(City Latitude, Seafood Restaurant)","(City Latitude, Shoe Repair)","(City Latitude

Grouping the dataframe with the most frequent venue.

In [39]:
df_cluster = df.join(venues_city, on='City')

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [40]:
df_cluster.drop(columns=['Latitude', 'Longitude'],inplace=True)

**Viewing Clusters**

Grouping the categories that most appear for each clusters.

Cluster 0

In [42]:
df_cluster.groupby(by='Cluster Labels').sum().transpose().sort_values(by=0, ascending=False).head()

Cluster Labels,0,1,2,3
PCI,11828.8,47074.2,16643.4,15620.1
"(City Latitude, Bakery)",9.0,7.0,14.0,7.0
"(Venue, Brazilian Restaurant)",9.0,8.0,17.0,13.0
"(Venue Longitude, Bakery)",9.0,7.0,14.0,7.0
"(Venue Longitude, Brazilian Restaurant)",9.0,8.0,17.0,13.0


Cluster 1

In [43]:
df_cluster.groupby(by='Cluster Labels').sum().transpose().sort_values(by=1, ascending=False).head()

Cluster Labels,0,1,2,3
PCI,11828.8,47074.2,16643.4,15620.1
"(City Longitude, Shopping Plaza)",1.0,8.0,0.0,0.0
"(Venue Latitude, Shopping Plaza)",1.0,8.0,0.0,0.0
"(Venue Latitude, Brazilian Restaurant)",9.0,8.0,17.0,13.0
"(Venue, Shopping Plaza)",1.0,8.0,0.0,0.0


Cluster 2

In [44]:
df_cluster.groupby(by='Cluster Labels').sum().transpose().sort_values(by=2, ascending=False).head()

Cluster Labels,0,1,2,3
PCI,11828.8,47074.2,16643.4,15620.1
"(City Latitude, Brazilian Restaurant)",9.0,8.0,17.0,13.0
"(Venue Latitude, Brazilian Restaurant)",9.0,8.0,17.0,13.0
"(Venue Longitude, Brazilian Restaurant)",9.0,8.0,17.0,13.0
"(Venue, Brazilian Restaurant)",9.0,8.0,17.0,13.0


Cluster 3

In [45]:
df_cluster.groupby(by='Cluster Labels').sum().transpose().sort_values(by=3, ascending=False).head()

Cluster Labels,0,1,2,3
PCI,11828.8,47074.2,16643.4,15620.1
"(City Latitude, Brazilian Restaurant)",9.0,8.0,17.0,13.0
"(City Longitude, Brazilian Restaurant)",9.0,8.0,17.0,13.0
"(Venue Latitude, Brazilian Restaurant)",9.0,8.0,17.0,13.0
"(Venue, Brazilian Restaurant)",9.0,8.0,17.0,13.0


6. Results:

The analysis shows that if the investor wants to invest in cities with high income he can invest in Restaurants. Given that Mall  equals the most frequent one marked by FourSquare users, it would be more interesting if he opened the restaurant inside a Mall.
